In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

2024-02-10 19:06:32.055477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 19:06:32.367933: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 19:06:33.247041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aaron/anaconda3/envs/tf/lib/
2024-02-10 19:06:33.247157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [2]:
train_data_dir = 'dataset'

In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


train_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    batch_size=32,
    class_mode='binary', 
    target_size=(224, 224)
)


Found 3001 images belonging to 2 classes.


In [4]:
# Load InceptionV3 model
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Unfreeze some top layers for fine-tuning
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Add more complexity to the custom fully connected layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)  # Added another dense layer
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = tf.keras.models.Model(base_model.input, x)

# Add EarlyStopping callback to monitor training loss and validation accuracy
early_stopping = EarlyStopping(monitor='acc', patience=5, restore_best_weights=True)

# Compile the model with the callback
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

# Train the model with early stopping using generator
inception_hist = model.fit(train_generator, 
                           steps_per_epoch=len(train_generator),
                           epochs=10,
                           callbacks=[early_stopping])

# Print the final accuracy
print("Final Accuracy: ", inception_hist.history['acc'][-1])

2024-02-10 19:06:34.653161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 19:06:34.667852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 19:06:34.668134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 19:06:34.668888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2024-02-10 19:06:45.579250: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-02-10 19:06:48.050457: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 19:06:48.051189: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 19:06:48.051214: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-02-10 19:06:48.051998: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 19:06:48.052155: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-02-10 19:06:49.098730: I tensorflow/stream_executor/cuda/c

94/94 [==============================] - 40s 287ms/step - loss: 0.4812 - acc: 0.8011
Epoch 2/10
94/94 [==============================] - 27s 282ms/step - loss: 0.3388 - acc: 0.8610
Epoch 3/10
94/94 [==============================] - 27s 284ms/step - loss: 0.2467 - acc: 0.8954
Epoch 4/10
94/94 [==============================] - 26s 271ms/step - loss: 0.1992 - acc: 0.9277
Epoch 5/10
94/94 [==============================] - 27s 281ms/step - loss: 0.1669 - acc: 0.9330
Epoch 6/10
94/94 [==============================] - 27s 282ms/step - loss: 0.1420 - acc: 0.9507
Epoch 7/10
94/94 [==============================] - 25s 269ms/step - loss: 0.1017 - acc: 0.9630
Epoch 8/10
94/94 [==============================] - 25s 266ms/step - loss: 0.1179 - acc: 0.9590
Epoch 9/10
94/94 [==============================] - 24s 256ms/step - loss: 0.0971 - acc: 0.9650
Epoch 10/10
94/94 [==============================] - 23s 247ms/step - loss: 0.0864 - acc: 0.9733
Final Accuracy:  0.97334223985672


In [5]:
# Save the model architecture to a JSON file
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an HDF5 file
model.save_weights("model_weights.h5")